In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pwd

'/content'

In [3]:
%cd drive/MyDrive/Github/ml_dl_tweets_sentiment/_10_52_mlflow_pytorch

/content/drive/MyDrive/Github/ml_dl_tweets_sentiment/_10_52_mlflow_pytorch


In [4]:
!pip install mlflow

     |████████████████████████████████| 16.5 MB 164 kB/s 
     |████████████████████████████████| 210 kB 53.1 MB/s 
     |████████████████████████████████| 596 kB 42.0 MB/s 
     |████████████████████████████████| 146 kB 57.2 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
     |████████████████████████████████| 181 kB 52.7 MB/s 
     |████████████████████████████████| 79 kB 6.6 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 78 kB 6.1 MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.16.4-py3-none-any.whl size=106877 sha256=e4e3a46720cff8518504b5643706ae90704c6c73a9b0d9ed8d2569ed92e14ca6
  Stored in directory: /root/.cache/pip/wheels/a2/a1/6d/fa1d22ea25ed8593887437fe1c7e00f6ef307fc240ccd4dc5c
Successfully built databricks-cli
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfu

In [5]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 4.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=80cf5c638badaeea07c31d4b870b6c9896375aa49ce6b7cffe8376f774128da6
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [6]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [7]:
from pyngrok import ngrok


In [8]:
!ngrok authtoken token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!cat /root/.ngrok2/ngrok.yml

In [10]:
ngrok.kill()


In [11]:
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)


In [ ]:
MLflow_UI_url = ngrok_tunnel.public_url

In [ ]:
print("MLflow UI ", MLflow_UI_url)


MLflow UI  https://841b-35-227-94-151.ngrok.io


In [13]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn

if __name__ == "__main__":

    mlflow.set_experiment(experiment_name="mlflow demo")
    
    training_data = pd.read_csv('../_09_47_local_model_store/storepurchasedata.csv')
    print("loaded training data")

    training_data.describe()
    mlflow.log_param("training percentage",60)

    mlflow.log_param("dataset shape",training_data.shape)
    
    X = training_data.iloc[:, :-1].values
    y = training_data.iloc[:,-1].values
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.4,random_state=0)
    
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    print("Completed Feature Scaling")
    
    from sklearn.neighbors import KNeighborsClassifier
    # minkowski is for ecledian distance
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    
    # Model training
    classifier.fit(X_train, y_train)
    print("Model trained")
   
    y_pred = classifier.predict(X_test)
    y_prob = classifier.predict_proba(X_test)[:,1]
    
    from sklearn.metrics import confusion_matrix
    
    cm = confusion_matrix(y_test, y_pred)
    
    from sklearn.metrics import accuracy_score
    
    model_accuracy = accuracy_score(y_test,y_pred)
    
    print(model_accuracy)
    
    mlflow.log_metric("accuracy", model_accuracy)
    mlflow.sklearn.log_model(classifier, "model")
    
    

2022/03/25 18:33:39 INFO mlflow.tracking.fluent: Experiment with name 'mlflow demo' does not exist. Creating a new experiment.


loaded training data
Completed Feature Scaling
Model trained
0.8125


In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

import mlflow
import mlflow.pytorch

if __name__ == "__main__":
    
    mlflow.set_experiment(experiment_name="mlflow pytorch demo")
    
    dataset = pd.read_csv('../_05_20_save_export_reload_pytorch_models/storepurchasedata_large.csv')
    
    dataset.describe()
    
    dataset.head()
    
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:,-1].values
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)
    
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    Xtrain_ = torch.from_numpy(X_train).float()
    Xtest_ = torch.from_numpy(X_test).float()
    
    Xtrain_
    
    ytrain_ = torch.from_numpy(y_train)
    ytest_ = torch.from_numpy(y_test)
    
    ytrain_
    
    Xtrain_.shape, ytrain_.shape
    
    Xtest_.shape, ytest_.shape
    
    input_size=2
    output_size=2
    hidden_size=10
    
    class Net(nn.Module):
       def __init__(self):
           super(Net, self).__init__()
           self.fc1 = torch.nn.Linear(input_size, hidden_size)
           self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
           self.fc3 = torch.nn.Linear(hidden_size, output_size)
    
    
       def forward(self, X):
           X = torch.relu((self.fc1(X)))
           X = torch.relu((self.fc2(X)))
           X = self.fc3(X)
    
           return F.log_softmax(X,dim=1)
    
    model = Net()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fn = nn.NLLLoss()
    
    epochs = 50
    
    for epoch in range(epochs):
      optimizer.zero_grad()
      Ypred = model(Xtrain_)
      loss = loss_fn(Ypred,  ytrain_)
      loss.backward()
      optimizer.step()
      print('Epoch',epoch, 'loss',loss.item())
    
    mlflow.end_run()
    with mlflow.start_run() as run:
        mlflow.log_param("epochs", 50)
        mlflow.pytorch.log_model(model, "models")
        mlflow.log_metric("loss", loss.item())



2022/03/25 18:33:54 INFO mlflow.tracking.fluent: Experiment with name 'mlflow pytorch demo' does not exist. Creating a new experiment.


Epoch 0 loss 0.6476587057113647
Epoch 1 loss 0.6316222548484802
Epoch 2 loss 0.6163710951805115
Epoch 3 loss 0.6016799211502075
Epoch 4 loss 0.5878496170043945
Epoch 5 loss 0.5738652348518372
Epoch 6 loss 0.5594539046287537
Epoch 7 loss 0.544397234916687
Epoch 8 loss 0.5285581350326538
Epoch 9 loss 0.5120475888252258
Epoch 10 loss 0.49519944190979004
Epoch 11 loss 0.47803157567977905
Epoch 12 loss 0.4603410065174103
Epoch 13 loss 0.4418550133705139
Epoch 14 loss 0.42257294058799744
Epoch 15 loss 0.4026099741458893
Epoch 16 loss 0.38222604990005493
Epoch 17 loss 0.3617686629295349
Epoch 18 loss 0.34156855940818787
Epoch 19 loss 0.32174623012542725
Epoch 20 loss 0.30232176184654236
Epoch 21 loss 0.2832203805446625
Epoch 22 loss 0.26487669348716736
Epoch 23 loss 0.24752309918403625
Epoch 24 loss 0.23123571276664734
Epoch 25 loss 0.21641814708709717
Epoch 26 loss 0.20295676589012146
Epoch 27 loss 0.19074758887290955
Epoch 28 loss 0.1798136830329895
Epoch 29 loss 0.17021417617797852
Epoch 3

2022/03/25 18:33:56 WARNING mlflow.utils.requirements_utils: Found torch version (1.10.0+cu111) contains a local version label (+cu111). MLflow logged a pip requirement for this package as 'torch==1.10.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/03/25 18:34:02 WARNING mlflow.utils.requirements_utils: Found torch version (1.10.0+cu111) contains a local version label (+cu111). MLflow logged a pip requirement for this package as 'torch==1.10.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
